In [20]:
import tensorflow as tf

In [21]:
# Session確立
sess = tf.InteractiveSession()

In [22]:
# データ読み込み
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [23]:
# placeholder
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# variable
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
#sess.run(tf.global_variables_initializer())

In [24]:
# 画像を28x28にreshape
# 4つめの引数はチャンネル数
x_image = tf.reshape(x, [-1,28,28,1])

In [25]:
# 第一層のweightsとbiasのvariable
#　5*5*1のフィルターを32個
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

# 第一層のconvolutionalとpool
#　1つずつ移動
#　2*2のフィルターを2個ずつ移動
#　出力は14*14*32の画像
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [26]:
# 第二層のweightsとbiasのvariable
#　5*5*32のフィルターを64個
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

# 第二層のconvolutionalとpool
#　1つずつ移動
#　2*2のフィルターを2個ずつ移動
#　出力は7*7*64の画像
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [27]:
# Relu層
#　weightsとbiasのvariable
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

#　ベクトルに再変換
#　第一層と第二層でreduceされてできた特徴に対してrelu
#　出力1024のベクトル
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [28]:
# Dropout層
#keep_prob: dropout率のplace_holder
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [29]:
# Softmax層
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [30]:
# 交差エントロピー層
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

In [31]:
# AdamOptimizerで最適化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [32]:
# accuracyを途中確認するための入れ物
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [33]:
# variable初期化
sess.run(tf.global_variables_initializer())

In [40]:
# accuracyを確認しつつtrain
accracies = []

#実行速度計測
import time
start = time.time()

for i in range(2000):
    batch_x, batch_y = mnist.train.next_batch(100)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_: batch_y, keep_prob: 1.0})
        accracies.append(train_accuracy)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        
    train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})

elapsed_time = time.time() - start

step 0, training accuracy 0.15
step 100, training accuracy 0.92
step 200, training accuracy 0.94
step 300, training accuracy 0.97
step 400, training accuracy 0.97
step 500, training accuracy 0.98
step 600, training accuracy 0.89
step 700, training accuracy 0.97
step 800, training accuracy 0.96
step 900, training accuracy 0.99
step 1000, training accuracy 0.95
step 1100, training accuracy 0.97
step 1200, training accuracy 0.97
step 1300, training accuracy 1
step 1400, training accuracy 0.98
step 1500, training accuracy 0.99
step 1600, training accuracy 0.97
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 1


In [19]:
#実行速度 about 8min
print(("finish_time: {0}".format(int(elapsed_time))) + " [sec]")

NameError: name 'elapsed_time' is not defined

In [ ]:
#10000枚の画像を投げるとキャパオーバーしてしまう（7000枚くらいが限界）
mnist.test.images.shape

In [ ]:
# test_accuracyを表示1　バッチ処理で制御
batch_tx, batch_ty = mnist.test.next_batch(7000)
print("test accuracy %g"%accuracy.eval(feed_dict={x:batch_tx, y_: batch_ty, keep_prob: 1.0}))
  #=> 0.9923

In [ ]:
# test_accuracyを表示2　スライスで制御
test_image = mnist.test.images[0:7000, :]
test_label = mnist.test.labels[0:7000, :]
print("test accuracy %g"%accuracy.eval(feed_dict={x: test_image, y_: test_label, keep_prob: 1.0}))